In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, DepReport, Dep
from typing import Dict
import pickle

with open(OUTPUT_PATH / "bcc.pkl", "rb") as f:
    report_data: Dict[str, Dict[Dep, DepReport]] = pickle.load(f)

In [17]:
from collections import defaultdict
from depsurf import DepKind
import pandas as pd

results = defaultdict(dict)

# tool_name = "biotop"
tool_name = "readahead"

reports = report_data[tool_name]
reports = {k: v for k, v in sorted(reports.items()) if k.kind != DepKind.STRUCT}
# pd.concat([r.df for r in reports.values()])

In [18]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

from depsurf import DepKind, Versions
from depsurf.output import save_fig


fig, ax = plt.subplots(figsize=(9, 9))
ax: plt.Axes

data = np.array(
    [[val.background_color for val in report.values] for report in reports.values()]
)

xlabels = (
    Versions.REGULAR.version_labels
    + Versions.ARCH.version_labels
    + Versions.REGULAR.pair_labels
    + Versions.ARCH.version_labels
)


ax.set_xticks(
    np.arange(data.shape[1]),
    labels=xlabels,
    rotation=90,
    fontsize=9,
)
ax.set_yticks(
    np.arange(data.shape[0]), labels=[d.name for d in reports.keys()], fontsize=9
)
ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
ax.imshow(data)

ax.grid(which="minor", color="white", linestyle="-", linewidth=0.1)
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.spines["top"].set_visible(True)
ax.spines["bottom"].set_visible(False)
ax.tick_params(which="minor", length=0)


def plot_secondary_ticks(ax: plt.Axes, axis_name, lengths, labels, pad, rotation=0):
    val = np.array(list(lengths))
    cum = np.cumsum(val)
    mid = cum - val / 2 - 0.5
    div = np.array([0, *cum][:-1]) - 0.5

    if axis_name == "x":
        axis = ax.secondary_xaxis("top").xaxis
    else:
        axis = ax.secondary_yaxis("left").yaxis

    # plot labels
    axis.set_ticks(mid, labels, rotation=rotation, verticalalignment="center")
    axis.set_tick_params(length=0, pad=pad)

    # plot ticks
    axis.set_ticks(div, labels=[], minor=True)
    axis.set_tick_params(length=pad + 10, width=0.5, which="minor")

    axline = ax.axvline if axis_name == "x" else ax.axhline
    for d in div:
        axline(d, color="black", lw=0.5, linestyle="--")


major_row = defaultdict(int)
for dep in reports.keys():
    kind = dep.kind
    if kind == DepKind.TRACEPOINT:
        kind = "TP"
    major_row[kind] += 1

plot_secondary_ticks(
    ax,
    "x",
    [
        len(Versions.REGULAR),
        len(Versions.ARCH),
        Versions.REGULAR.num_pairs,
        Versions.ARCH.num_pairs,
    ],
    ["Version", "Arch", "Version Diff", "Arch Diff\nw/ x86"],
    pad=60,
)

plot_secondary_ticks(
    ax,
    "y",
    major_row.values(),
    major_row.keys(),
    pad=140,
    rotation=90,
)


for i, report in enumerate(reports.values()):
    for j, val in enumerate(report.values):
        ax.text(
            j,
            i,
            val.text,
            ha="center",
            va="center",
            color=val.text_color,
            fontsize=8,
        )

# enums = list(DepStatusEnum) + [
#     DepDeltaEnum.REMOVE,
#     DepDeltaEnum.ADD,
#     DepDeltaEnum.CHANGE,
# ]


# legends = [(f"{e.symbol}: {e.name}" if e.symbol else e.name, e.color) for e in enums]
# ax.legend(
#     handles=[plt.Rectangle((0, 0), 1, 1, fc=color) for text, color in legends],
#     labels=[text for text, color in legends],
#     loc="upper center",
#     bbox_to_anchor=(0.375, 1.275),
#     ncol=len(enums),
#     handletextpad=0.5,
#     columnspacing=1,
#     handlelength=1,
#     handleheight=1.2,
#     # loc="upper left",
#     # bbox_to_anchor=(-0.285, 1.225),
#     # ncol = 2,
#     # handletextpad=0.2,
#     # columnspacing=0.5,
#     # handlelength=0.7,
#     # handleheight=0.7,
#     fontsize=8,
# )


save_fig(fig, tool_name, close=True)

[          mpl.py:79 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/readahead.pdf
